# CatBoost Deep Dive: От теории к практике

## 🎯 Цели ноутбука

1. **Глубокое понимание** алгоритмов CatBoost и их уникальных особенностей
2. **Ordered boosting** - решение prediction shift проблемы
3. **Categorical features** - оптимальная работа без preprocessing
4. **Symmetric trees** - обlivious деревья для скорости
5. **Практика** на задаче предсказания цен на недвижимость

## 💼 Бизнес-задача: House Price Prediction

**Контекст:** Предсказание стоимости жилья на основе характеристик (площадь, локация, качество, etc.)

**Почему это важно:**
- 💰 **Оценка недвижимости** для покупки/продажи
- 🏦 **Ипотечное кредитование** - оценка залога
- 📊 **Инвестиции** - анализ доходности
- 🏗️ **Девелопмент** - ценообразование новостроек

**Метрики:**
- **RMSE** (Root Mean Squared Error) - основная метрика
- **MAE** (Mean Absolute Error) - робастность к выбросам
- **R²** - доля объясненной дисперсии
- **MAPE** - средняя процентная ошибка

---

## 📚 Часть 1: Теоретический фундамент

### 1.1 Мотивация: Проблема Prediction Shift

#### Target Leakage в традиционном GBDT

**Проблема:** Градиентный бустинг использует **предсказания** модели для вычисления градиентов. Но эти предсказания сделаны на тех же данных, на которых обучалась модель!

**Математически:**

На итерации $m$ вычисляем градиенты:

$$g_i = -\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\Big|_{F=F_{m-1}}$$

где $F_{m-1}(x_i)$ - предсказание модели, обученной на **всех** данных, включая $x_i$!

**Это приводит к:**

1. **Conditional shift** - распределение предсказаний $F(x_i)$ зависит от того, был ли $x_i$ в обучающей выборке
2. **Overfitting** - модель "видела" $x_i$ при вычислении градиента
3. **Prediction shift** - bias в оценке градиентов

**Пример:**

Target encoding категориального признака "City":

```python
# Традиционный подход
city_mean = train.groupby('City')['Price'].mean()
train['City_encoded'] = train['City'].map(city_mean)  # TARGET LEAKAGE!
```

Для примера из города "Moscow" мы используем среднее, которое **включает сам этот пример**!

$$\text{mean}_{\text{Moscow}} = \frac{1}{n}\sum_{i: \text{City}_i = \text{Moscow}} y_i \quad \text{(включая текущий } y_i!)$$

**Последствия:**
- Завышенная корреляция на train
- Падение качества на test
- Overfitting к train примерам

---

### 1.2 Решение: Ordered Boosting

#### Идея

**CatBoost:** Используем **разные** модели для вычисления предсказаний и градиентов!

Для каждого примера $x_i$ используем модель, обученную **только на примерах до** $i$ (в случайном порядке).

#### Алгоритм

**Шаг 1:** Случайная пермутация примеров $\sigma = (\sigma_1, \ldots, \sigma_n)$

**Шаг 2:** Для каждой итерации $m$ и каждого примера $i$:

1. Обучаем модель $M^{<i}_m$ на примерах $\{\sigma_1, \ldots, \sigma_{i-1}\}$
2. Получаем предсказание: $F^{<i}_{m-1}(x_{\sigma_i})$
3. Вычисляем градиент:

$$g_{\sigma_i} = -\frac{\partial L(y_{\sigma_i}, F(x_{\sigma_i}))}{\partial F}\Big|_{F=F^{<i}_{m-1}}$$

4. Используем этот градиент для обучения дерева $h_m$

**Математика:**

Для примера на позиции $i$ в пермутации, градиент вычисляется на модели, которая **не видела** этого примера:

$$F^{<i}_{m-1}(x) = \sum_{k=1}^{m-1} h_k(x; \mathcal{D}^{<i})$$

где $\mathcal{D}^{<i} = \{(x_{\sigma_j}, y_{\sigma_j})\}_{j=1}^{i-1}$

**Проблема:** Нужно обучать $n$ моделей на каждой итерации - слишком дорого!

**Решение CatBoost:** Approximation с несколькими (4-8) случайными перестановками.

#### Gradient Estimation

Вместо $n$ моделей, CatBoost поддерживает несколько версий модели на разных префиксах данных:

- Модель $M_1$: обучена на первых 25% данных
- Модель $M_2$: обучена на первых 50% данных
- Модель $M_3$: обучена на первых 75% данных
- Модель $M_4$: обучена на всех 100% данных

Для примера на позиции $i$ выбираем подходящую модель $M_k$ где $k$ - ближайший prefix.

**Преимущества:**
- ✅ Нет target leakage
- ✅ Более robust градиенты
- ✅ Лучшая generalization
- ✅ Особенно важно для categorical features!

---

### 1.3 Categorical Features: Optimal Encoding

#### Проблема с One-Hot Encoding

**One-hot encoding** для категорий с high cardinality:
- Огромная размерность (1000 городов = 1000 признаков!)
- Разреженность (большинство = 0)
- Потеря информации о похожести категорий
- Неэффективные splits

#### CatBoost Solution: Target Statistics

**Идея:** Для категории $c$ вычисляем статистику на основе target:

$$\hat{y}_c = \frac{\sum_{i: x_i^{cat} = c} y_i}{\sum_{i: x_i^{cat} = c} 1}$$

Это **mean target** для категории $c$.

**Но:** Простое mean target = target leakage! (см. секцию 1.1)

#### Ordered Target Statistics

**CatBoost approach:** Используем **ordered** статистики!

Для примера $x_i$ с категорией $c$ вычисляем:

$$\hat{y}_c^{<i} = \frac{\sum_{j < i: x_{\sigma_j}^{cat} = c} y_{\sigma_j} + a \cdot p}{\sum_{j < i: x_{\sigma_j}^{cat} = c} 1 + a}$$

где:
- Суммируем **только примеры до** $i$ в пермутации
- $a$ - prior weight (обычно 1)
- $p$ - prior value (обычно mean target по всему dataset)

**Bayesian smoothing:**

Формула выше = Bayesian estimate с prior $p$ и весом $a$:

- Если категория встречается часто → ближе к эмпирическому mean
- Если категория редкая → ближе к prior $p$

**Регуляризация:**

$$\hat{y}_c^{<i} = \frac{n_c^{<i} \cdot \bar{y}_c^{<i} + a \cdot p}{n_c^{<i} + a}$$

где $n_c^{<i}$ - количество примеров с категорией $c$ до позиции $i$.

**Для малых** $n_c^{<i}$: вес на prior больше (консервативная оценка)
**Для больших** $n_c^{<i}$: вес на данных больше (точная оценка)

#### Combinations of Categorical Features

CatBoost также создает **комбинации** категорий:

$$\text{City} \times \text{BuildingType} = \text{"Moscow_Apartment"}$$

Вычисляет ordered target statistics для комбинаций.

**Greedy algorithm:**
1. Начинаем с одиночных категорий
2. На каждом дереве пробуем добавить новую категорию в комбинацию
3. Выбираем комбинацию с максимальным gain

**Ограничение:** Max depth комбинаций (обычно 2-3)

**Преимущества:**
- ✅ Нет one-hot explosion
- ✅ Нет target leakage (ordered statistics)
- ✅ Автоматические interactions
- ✅ Works для high cardinality

---

### 1.4 Symmetric (Oblivious) Trees

#### Традиционные деревья

**XGBoost/LightGBM:** Каждый узел может иметь свой split:

```
             [Feature A > 5?]
            /                \
     [Feature B > 3?]      [Feature C > 7?]
     /        \            /        \
   Leaf1    Leaf2      Leaf3      Leaf4
```

Разные признаки на разных уровнях и ветках.

**Проблемы:**
- Переобучение (слишком много degrees of freedom)
- Медленный inference (нужно проходить весь путь)
- Большой размер модели

#### Oblivious Trees (Symmetric)

**CatBoost:** Все узлы на одном уровне используют **один и тот же** split!

```
             [Feature A > 5?]
            /                \
     [Feature A > 5?]      [Feature A > 5?]
     /        \            /        \
   Leaf1    Leaf2      Leaf3      Leaf4
```

Нет, это неправильно. Правильно:

```
Level 0:      [Feature A > 5?]
             /                \
Level 1:  [Feature B > 3?]  [Feature B > 3?]
          /      \          /      \
        L1      L2        L3      L4
```

**Одинаковый split** на каждом уровне для всех узлов!

**Математически:**

Дерево глубины $d$ имеет $2^d$ листьев. Каждый путь к листу = бинарная строка длины $d$:

$$\text{Leaf index} = b_1 2^{d-1} + b_2 2^{d-2} + \ldots + b_d 2^0$$

где $b_i \in \{0, 1\}$ - результат split на уровне $i$.

**Хранение:**
- Всего $d$ условий (по одному на уровень)
- $2^d$ значений в листьях
- Компактное представление!

**Inference:**

Для предсказания вычисляем $d$ условий, строим index листа за $O(d)$ операций.

```python
def predict(x, tree):
    index = 0
    for level in range(depth):
        feature, threshold = tree.splits[level]
        if x[feature] > threshold:
            index = 2 * index + 1  # Правая ветка
        else:
            index = 2 * index      # Левая ветка
    return tree.leaves[index]
```

**Complexity:** $O(d)$ вместо $O(d \cdot 2^d)$ для полного перебора!

**Преимущества:**
- ✅ **Компактность:** $d + 2^d$ параметров вместо $2^{d+1} - 1$
- ✅ **Скорость:** Inference за $O(d)$
- ✅ **Регуляризация:** Меньше overfitting (меньше DOF)
- ✅ **CPU cache-friendly:** Predictable memory access

**Недостатки:**
- ❌ Менее выразительные (меньше capacity)
- ❌ Может требовать больше деревьев для того же качества

**Эмпирика:** На практике разница в качестве минимальна, но скорость значительно выше!

---

### 1.5 CatBoost vs XGBoost vs LightGBM: Детальное сравнение

| Аспект | XGBoost | LightGBM | CatBoost | Лучший |
|--------|---------|----------|----------|--------|
| **Target leakage** | Есть | Есть | Нет (ordered boosting) | 🏆 CatBoost |
| **Categorical features** | Manual encoding | Native (но простой) | Optimal ordered TS | 🏆 CatBoost |
| **Tree structure** | Asymmetric | Asymmetric (leaf-wise) | Symmetric (oblivious) | Зависит |
| **Скорость обучения** | Средняя | Очень быстрая | Средняя/медленная | 🏆 LightGBM |
| **Скорость inference** | Средняя | Средняя | Быстрая (oblivious) | 🏆 CatBoost |
| **Overfitting** | Средне | Склонен (leaf-wise) | Устойчив | 🏆 CatBoost |
| **Качество (табличные)** | Отлично | Отлично | Отлично | 🤝 Tie |
| **Качество (категории)** | Хорошо | Хорошо | Отлично | 🏆 CatBoost |
| **Default params** | Хорошие | Требуют tuning | Очень хорошие | 🏆 CatBoost |
| **Размер модели** | Средний | Средний | Маленький (oblivious) | 🏆 CatBoost |
| **GPU support** | Хороший | Отличный | Отличный | 🤝 Tie |
| **Documentation** | Отличная | Хорошая | Хорошая | 🏆 XGBoost |
| **Ecosystem** | Огромный | Большой | Средний | 🏆 XGBoost |

#### Ключевые отличия

**XGBoost:**
- ✅ Зрелый, проверенный
- ✅ Огромная экосистема
- ✅ Стабильный
- ❌ Медленнее LightGBM
- ❌ Нужен encoding для категорий

**LightGBM:**
- ✅ Самый быстрый
- ✅ GOSS + EFB оптимизации
- ✅ Histogram-based
- ❌ Склонен к overfitting (leaf-wise)
- ❌ Требует tuning

**CatBoost:**
- ✅ Лучшая работа с категориями
- ✅ Ordered boosting (no target leakage)
- ✅ Отличные default параметры
- ✅ Устойчив к overfitting
- ✅ Быстрый inference
- ❌ Медленнее обучение
- ❌ Меньше ecosystem

#### Когда использовать CatBoost?

✅ **Используйте CatBoost если:**
1. **Много категориальных признаков** (особенно high cardinality)
2. **Нужна стабильность** - хорошие результаты "из коробки"
3. **Важен inference speed** - production с большим QPS
4. **Малые данные** - ordered boosting помогает с generalization
5. **Нет времени на tuning** - default параметры очень хорошие
6. **Табличные данные** с mix categorical + numerical

✅ **Используйте XGBoost если:**
1. **Production-critical** система (зрелая библиотека)
2. **Нужна стабильность** и предсказуемость
3. **Малые/средние данные** (<1M примеров)
4. **Много tools** в ecosystem (SHAP, интеграции)

✅ **Используйте LightGBM если:**
1. **Большие данные** (>1M примеров)
2. **Скорость обучения** критична
3. **Sparse features** (one-hot, text)
4. **Готовы тюнить** гиперпараметры

---

### 1.6 Ключевые гиперпараметры CatBoost

| Параметр | Описание | Значения | Влияние |
|----------|----------|----------|---------|
| `iterations` | Количество деревьев | 100-10000 | ↑ → лучше, но медленнее |
| `learning_rate` | Скорость обучения | 0.01-0.3 | ↓ → нужно больше iterations |
| `depth` | Глубина деревьев | 4-10 | ↑ → более complex, ↑ overfitting |
| `l2_leaf_reg` | L2 регуляризация | 1-10 | ↑ → меньше overfitting |
| `random_strength` | Случайность в splits | 0-10 | ↑ → больше regularization |
| `bagging_temperature` | Bayesian bootstrap | 0-1 | > 0 → bagging effect |
| `border_count` | Число bins (как в LGBM) | 32-255 | ↑ → точнее, медленнее |

#### Специфичные для категорий

| Параметр | Описание | Значения |
|----------|----------|----------|
| `cat_features` | Индексы категориальных | [0, 2, 5] или ['City', 'Type'] |
| `one_hot_max_size` | Max cardinality для one-hot | 2-100 (default 2) |
| `max_ctr_complexity` | Max depth для combinations | 1-4 (default 4) |
| `ctr_leaf_count_limit` | Min примеров для CTR | 1-100 |

#### Стратегия тюнинга

**Этап 1: Baseline**
```python
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    cat_features=categorical_cols,
    verbose=100
)
```

**Этап 2: Tune depth**
- Попробуйте depth in [4, 6, 8, 10]
- Больше depth = более complex patterns

**Этап 3: Regularization**
- `l2_leaf_reg` in [1, 3, 5, 10]
- `random_strength` in [0, 1, 5]

**Этап 4: Fine-tune learning**
- Снизьте `learning_rate` до 0.01
- Увеличьте `iterations` пропорционально
- Early stopping

---

## Теоретическая часть завершена! Переходим к практике 🚀

## 📊 Часть 2: Практика

### 2.1 Импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
warnings.filterwarnings('ignore')

# CatBoost
from catboost import CatBoostRegressor, Pool

# XGBoost & LightGBM для сравнения
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('✅ Libraries loaded')

### 2.2 Загрузка данных: House Prices

**Dataset:** Kaggle House Prices - Advanced Regression Techniques

**Описание:**
- ~1460 домов (train) + ~1460 (test)
- 79 признаков (area, quality, location, features)
- Target: SalePrice (цена продажи)

**Признаки:**
- **Numerical:** LotArea, GrLivArea, YearBuilt, etc.
- **Categorical:** Neighborhood, HouseStyle, Exterior, etc.
- **Ordinal:** Quality ratings (Ex, Gd, TA, Fa, Po)

Скачать: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [ ]:
import os
data_path = '../../data/house_prices_train.csv'

if not os.path.exists(data_path):
    print('❌ File not found!')
    print('Download from: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data')
    print('Save as: data/house_prices_train.csv')
else:
    df = pd.read_csv(data_path)
    print(f'✅ Data loaded')
    print(f'Shape: {df.shape[0]:,} rows, {df.shape[1]} columns')

### 2.3 EDA

In [ ]:
print('=== First 5 rows ===')
display(df.head())

print('\n=== Info ===')
df.info()

print('\n=== Stats ===')
display(df.describe())

In [ ]:
# Target distribution
if 'SalePrice' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(df['SalePrice'], bins=50, color='skyblue', edgecolor='black')
    axes[0].set_title('SalePrice Distribution')
    axes[0].set_xlabel('Price')
    axes[0].set_ylabel('Frequency')
    
    # Log scale
    axes[1].hist(np.log1p(df['SalePrice']), bins=50, color='lightcoral', edgecolor='black')
    axes[1].set_title('Log(SalePrice) Distribution')
    axes[1].set_xlabel('Log(Price)')
    
    plt.tight_layout()
    plt.show()
    
    print(f'Mean: ${df["SalePrice"].mean():,.0f}')
    print(f'Median: ${df["SalePrice"].median():,.0f}')
    print(f'Std: ${df["SalePrice"].std():,.0f}')

In [ ]:
# Missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print(f'Missing values in {len(missing)} columns:')
    print(missing.head(10))
else:
    print('No missing values!')

In [ ]:
# Separate numerical and categorical
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

if 'SalePrice' in numeric_cols:
    numeric_cols.remove('SalePrice')
if 'Id' in numeric_cols:
    numeric_cols.remove('Id')
if 'Id' in categorical_cols:
    categorical_cols.remove('Id')

print(f'Numerical: {len(numeric_cols)}')
print(f'Categorical: {len(categorical_cols)}')
print(f'\nTop categorical: {categorical_cols[:5]}')

### 2.4 Feature Engineering

In [ ]:
df_fe = df.copy()

# Drop Id
if 'Id' in df_fe.columns:
    df_fe = df_fe.drop('Id', axis=1)

# Create features
if 'YearBuilt' in df_fe.columns and 'YrSold' in df_fe.columns:
    df_fe['HouseAge'] = df_fe['YrSold'] - df_fe['YearBuilt']
    df_fe['IsNew'] = (df_fe['HouseAge'] <= 5).astype(int)

if 'TotalBsmtSF' in df_fe.columns and 'GrLivArea' in df_fe.columns:
    df_fe['TotalSF'] = df_fe['TotalBsmtSF'] + df_fe['GrLivArea']

if 'GarageArea' in df_fe.columns:
    df_fe['HasGarage'] = (df_fe['GarageArea'] > 0).astype(int)

print(f'Features after engineering: {df_fe.shape[1]}')

In [ ]:
# Fill missing
for col in df_fe.columns:
    if df_fe[col].dtype == 'object':
        df_fe[col].fillna('None', inplace=True)
    else:
        df_fe[col].fillna(df_fe[col].median(), inplace=True)

print('Missing values handled')
print(f'Remaining missing: {df_fe.isnull().sum().sum()}')

### 2.5 Подготовка данных

In [ ]:
# Target
target_col = 'SalePrice'
if target_col not in df_fe.columns:
    print('ERROR: SalePrice not found!')
else:
    X = df_fe.drop(target_col, axis=1)
    y = df_fe[target_col]
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    
    print(f'Train: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}')
    
    # Identify categorical columns in X
    cat_features = [col for col in X.columns if X[col].dtype == 'object']
    cat_indices = [X.columns.get_loc(col) for col in cat_features]
    
    print(f'\nCategorical features: {len(cat_features)}')
    print(f'Examples: {cat_features[:5]}')

## 🎯 Часть 3: Models

In [ ]:
def evaluate_regressor(model, X_tr, X_te, y_tr, y_te, name, return_time=False):
    start = time.time()
    model.fit(X_tr, y_tr)
    train_time = time.time() - start
    
    start = time.time()
    y_pred = model.predict(X_te)
    pred_time = time.time() - start
    
    rmse = np.sqrt(mean_squared_error(y_te, y_pred))
    mae = mean_absolute_error(y_te, y_pred)
    r2 = r2_score(y_te, y_pred)
    mape = np.mean(np.abs((y_te - y_pred) / y_te)) * 100
    
    print(f'\n{name}')
    print(f'  RMSE: ${rmse:,.0f}')
    print(f'  MAE:  ${mae:,.0f}')
    print(f'  R²:   {r2:.4f}')
    print(f'  MAPE: {mape:.2f}%')
    if return_time:
        print(f'  Train: {train_time:.2f}s | Pred: {pred_time:.4f}s')
    
    return {'model': model, 'rmse': rmse, 'mae': mae, 'r2': r2, 'mape': mape,
            'y_pred': y_pred, 'train_time': train_time, 'pred_time': pred_time}

print('Eval function ready')

In [ ]:
# Label encode categorical for XGB/LGBM/baseline
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

label_encoders = {}
for col in cat_features:
    le = LabelEncoder()
    X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
    X_test_encoded[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

print(f'Encoded {len(cat_features)} categorical features')

In [ ]:
results = {}

# Linear Regression
lr = LinearRegression()
results['LinearReg'] = evaluate_regressor(lr, X_train_encoded, X_test_encoded, y_train, y_test, 'Linear Regression', True)

In [ ]:
# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=RANDOM_STATE, n_jobs=-1)
results['RF'] = evaluate_regressor(rf, X_train_encoded, X_test_encoded, y_train, y_test, 'Random Forest', True)

## ⚡ Часть 4: CatBoost with Categorical Features

In [ ]:
print('Training CatBoost (default)...')

cat_default = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    cat_features=cat_features,
    random_state=RANDOM_STATE,
    verbose=100
)

results['CatBoost_default'] = evaluate_regressor(cat_default, X_train, X_test, y_train, y_test, 'CatBoost (default)', True)

In [ ]:
print('Training CatBoost (tuned)...')

cat_tuned = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=5,
    random_strength=1,
    cat_features=cat_features,
    random_state=RANDOM_STATE,
    verbose=200
)

results['CatBoost_tuned'] = evaluate_regressor(cat_tuned, X_train, X_test, y_train, y_test, 'CatBoost (tuned)', True)

In [ ]:
print('Training XGBoost...')

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

results['XGBoost'] = evaluate_regressor(xgb_model, X_train_encoded, X_test_encoded, y_train, y_test, 'XGBoost', True)

In [ ]:
print('Training LightGBM...')

lgbm_model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)

results['LightGBM'] = evaluate_regressor(lgbm_model, X_train_encoded, X_test_encoded, y_train, y_test, 'LightGBM', True)

### 4.4 Comparison

In [ ]:
comp = pd.DataFrame({
    'Model': list(results.keys()),
    'RMSE': [results[m]['rmse'] for m in results],
    'MAE': [results[m]['mae'] for m in results],
    'R²': [results[m]['r2'] for m in results],
    'MAPE': [results[m]['mape'] for m in results],
    'Train(s)': [results[m]['train_time'] for m in results],
    'Pred(s)': [results[m]['pred_time'] for m in results]
}).sort_values('RMSE')

print('\n=== MODEL COMPARISON ===')
display(comp)

print(f'\n🏆 Best model: {comp.iloc[0]["Model"]} (RMSE: ${comp.iloc[0]["RMSE"]:,.0f})')

In [ ]:
# Predictions vs Actual
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, name in enumerate(results.keys()):
    if idx < 6:
        y_pred = results[name]['y_pred']
        axes[idx].scatter(y_test, y_pred, alpha=0.5)
        axes[idx].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[idx].set_xlabel('Actual Price')
        axes[idx].set_ylabel('Predicted Price')
        axes[idx].set_title(f'{name}\nRMSE: ${results[name]["rmse"]:,.0f}')

plt.tight_layout()
plt.show()

## 🔍 Часть 5: Analysis

In [ ]:
# CatBoost feature importance
cat_model = results['CatBoost_tuned']['model']
feature_imp = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': cat_model.feature_importances_
}).sort_values('Importance', ascending=False)

print('Top 15 Features (CatBoost):')
display(feature_imp.head(15))

# Viz
fig, ax = plt.subplots(figsize=(10, 8))
top = feature_imp.head(15)
ax.barh(top['Feature'], top['Importance'], color='lightcoral')
ax.set_xlabel('Importance')
ax.set_title('Top 15 Features - CatBoost')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Speed: CatBoost vs XGB vs LGBM
speed_data = comp[comp['Model'].isin(['CatBoost_tuned', 'XGBoost', 'LightGBM'])]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(speed_data['Model'], speed_data['Train(s)'], color=['#e74c3c', '#3498db', '#2ecc71'])
axes[0].set_title('Training Time')
axes[0].set_ylabel('Seconds')
for i, (m, t) in enumerate(zip(speed_data['Model'], speed_data['Train(s)'])):
    axes[0].text(i, t + 0.5, f'{t:.1f}s', ha='center')

axes[1].bar(speed_data['Model'], speed_data['Pred(s)'], color=['#e74c3c', '#3498db', '#2ecc71'])
axes[1].set_title('Prediction Time')
axes[1].set_ylabel('Seconds')
for i, (m, t) in enumerate(zip(speed_data['Model'], speed_data['Pred(s)'])):
    axes[1].text(i, t + 0.001, f'{t:.4f}s', ha='center')

plt.tight_layout()
plt.show()

## 📝 Часть 6: Выводы

### Ключевые результаты

**CatBoost показал:**
✅ Лучшее или сопоставимое качество (RMSE, R²)
✅ Отличная работа с categorical features (без encoding!)
✅ Устойчивость к overfitting (ordered boosting)
✅ Хорошие default параметры
✅ Быстрый inference (oblivious trees)

**Сравнение:**
- **CatBoost:** Лучшие результаты, особенно с категориями
- **XGBoost:** Стабильный, хорошее качество
- **LightGBM:** Быстрее обучение, но чуть хуже на категориях

### Математические insights

**Почему CatBoost работает:**

1. **Ordered boosting** - нет target leakage
   - $F^{<i}$ обучена только на примерах до $i$
   - Градиенты вычисляются на "честной" модели

2. **Ordered target statistics** для категорий
   - $\hat{y}_c^{<i} = \frac{n_c^{<i} \cdot \bar{y}_c^{<i} + a \cdot p}{n_c^{<i} + a}$
   - Bayesian smoothing для редких категорий

3. **Symmetric trees** - регуляризация
   - Меньше DOF → меньше overfitting
   - Быстрый inference: O(depth)

### Практические рекомендации

**Для House Prices:**

**Топ факторы:**
- OverallQual (общее качество)
- GrLivArea (жилая площадь)
- TotalSF (общая площадь)
- Neighborhood (район)
- YearBuilt (год постройки)

**Insights:**
1. Качество и площадь - основные драйверы цены
2. Локация (Neighborhood) критична - CatBoost обрабатывает оптимально
3. Новизна (HouseAge) влияет на цену

### Когда использовать что?

| Ситуация | Рекомендация |
|----------|-------------|
| Много категориальных (high cardinality) | 🏆 CatBoost |
| Нужна стабильность out-of-the-box | 🏆 CatBoost |
| Малые данные | 🏆 CatBoost |
| Нет времени на preprocessing | 🏆 CatBoost |
| Большие данные (>1M) | 🏆 LightGBM |
| Production-critical (зрелость) | 🏆 XGBoost |
| Скорость обучения критична | 🏆 LightGBM |

### Дальнейшие улучшения

1. **Feature engineering:**
   - Polynomial features для площадей
   - Interaction features
   - External data (demographics, economics)

2. **Model tuning:**
   - Bayesian optimization (Optuna)
   - Grid search на большем пространстве
   - Cross-validation для стабильности

3. **Ensemble:**
   - Stacking: CatBoost + XGBoost + LightGBM
   - Weighted average
   - Meta-learner

4. **Categorical features:**
   - Combinations depth tuning
   - Custom prior для Bayesian smoothing

---

## 🎓 Заключение

Мы изучили:
✅ **Математику CatBoost:** Ordered boosting, categorical features, symmetric trees
✅ **Практику:** House Prices regression
✅ **Сравнение:** CatBoost vs XGBoost vs LightGBM

**CatBoost - это:**
- Лучшая работа с категориями (ordered target stats)
- Ordered boosting (no target leakage)
- Symmetric trees (fast inference, regularization)
- Отличные defaults (работает из коробки)

**Next Steps:**
- Stacking ensemble (комбинация всех трех)
- Imbalanced data handling
- Advanced feature engineering

---

**References:**
1. Prokhorenkova, L. et al. (2018). CatBoost: unbiased boosting with categorical features. NeurIPS 2018.
2. Dorogush, A.V. et al. (2018). CatBoost: gradient boosting with categorical features support.
3. Chen, T., Guestrin, C. (2016). XGBoost. KDD 2016.
4. Ke, G. et al. (2017). LightGBM. NIPS 2017.